Choose GPU

In [1]:
import os
KAGGLE = True
if os.name == "nt":
    KAGGLE = False
print(f"KAGGLE: {KAGGLE}")
if not KAGGLE:
    os.environ["CUDA_VISIBLE_DEVICES"] = "0"

KAGGLE: False


In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from glob import glob
tf.compat.v1.disable_eager_execution()

In [3]:
INPUT_DIR = "../input/g2net-gravitational-wave-detection"
tf.keras.mixed_precision.set_global_policy("mixed_float16")
BATCH_SIZE = 256

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 3090, compute capability 8.6


In [4]:
def load_dataset(files):
    dataset = tf.data.TFRecordDataset(files)

    def _parse_function(example_proto):
        keys_to_feature = {"X":tf.io.FixedLenFeature(([4096,3]),tf.float32),
                           "y":tf.io.FixedLenFeature((), tf.int64, default_value=0)}
        parsed_features = tf.io.parse_single_example(example_proto, keys_to_feature)
        return parsed_features["X"], parsed_features["y"]

    # parse the record into tensors.
    dataset = dataset.map(_parse_function)

    # shuffle the dataset
    dataset = dataset.shuffle(buffer_size=10000)

    # Repeat the input infinitely
    dataset = dataset.repeat()


    # Generate batches
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(-1)
    return dataset

In [5]:
train_files = []
for i in range(3):
    train_files.append(f"train_{i}.tfrec")

train_ds = load_dataset(train_files)
val_files = ["train_3.tfrec"]
val_ds = load_dataset(val_files)

In [6]:
import gc
gc.collect()

21

In [21]:

from tensorflow.keras import Sequential, layers
model = Sequential([
    layers.Conv1D(filters=32, kernel_size=128, padding="causal",input_shape=[4096,3]),
    layers.Dropout(0.5),
    layers.Conv1D(filters=32, kernel_size=16, padding="causal"),
    layers.MaxPool1D(pool_size=4),
    layers.Activation("relu"),
    
    layers.Conv1D(filters=64, kernel_size=8, padding="causal"),
    layers.Conv1D(filters=128, kernel_size=8, padding="causal"),
    layers.Conv1D(filters=128, kernel_size=8, padding="causal"),
    layers.MaxPool1D(pool_size=4),
    layers.Dropout(0.4),
    layers.Activation("relu"),
    
    layers.Conv1D(filters=128, kernel_size=8, padding="causal"),
    layers.MaxPool1D(pool_size=4),
    layers.Activation("relu"),
    
    layers.Conv1D(filters=256, kernel_size=8, padding="causal"),
    layers.MaxPool1D(pool_size=4),
    layers.Activation("relu"),
    
    layers.Flatten(),
    layers.Dense(128, activation="relu"),
    layers.Dense(64, activation="relu"),
    layers.Dropout(0.5),
    layers.Dense(1),
    layers.Activation("sigmoid", dtype="float32")
])
model.compile(
    tf.keras.optimizers.Adam(),
    loss="binary_crossentropy",
    metrics=["AUC"]
)
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_13 (Conv1D)           (None, 4096, 32)          12320     
_________________________________________________________________
dropout_4 (Dropout)          (None, 4096, 32)          0         
_________________________________________________________________
conv1d_14 (Conv1D)           (None, 4096, 32)          16416     
_________________________________________________________________
max_pooling1d_8 (MaxPooling1 (None, 1024, 32)          0         
_________________________________________________________________
activation_10 (Activation)   (None, 1024, 32)          0         
_________________________________________________________________
conv1d_15 (Conv1D)           (None, 1024, 64)          16448     
_________________________________________________________________
conv1d_16 (Conv1D)           (None, 1024, 128)        

In [22]:
steps_per_epoch = 560000 // 16 * len(train_files) // BATCH_SIZE
validation_steps = 560000 // 16 * len(val_files) // BATCH_SIZE

In [23]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss", factor=0.2,
    patience=5, min_lr = 0.00001,
    verbose=1
)
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    mode="min",
    patience=10
)
callbacks=[reduce_lr, early_stop]

In [ ]:
history = model.fit(train_ds, validation_data = val_ds, epochs = 40, shuffle=True,
                    steps_per_epoch = steps_per_epoch, validation_steps=validation_steps,
                    verbose=1, callbacks=[callbacks])

Train on 410 steps, validate on 136 steps
Epoch 1/40
410/410 [==============================] - 31s 71ms/step - batch: 204.5000 - size: 1.0000 - loss: 0.5606 - AUC: 0.7537 - val_loss: 0.5202 - val_AUC: 0.7911 - lr: 0.0010
Epoch 2/40
410/410 [==============================] - 29s 72ms/step - batch: 204.5000 - size: 1.0000 - loss: 0.5076 - AUC: 0.8008 - val_loss: 0.5010 - val_AUC: 0.8109 - lr: 0.0010
Epoch 3/40
410/410 [==============================] - 29s 72ms/step - batch: 204.5000 - size: 1.0000 - loss: 0.4971 - AUC: 0.8079 - val_loss: 0.4873 - val_AUC: 0.8167 - lr: 0.0010
Epoch 4/40
410/410 [==============================] - 30s 73ms/step - batch: 204.5000 - size: 1.0000 - loss: 0.4934 - AUC: 0.8105 - val_loss: 0.4892 - val_AUC: 0.8155 - lr: 0.0010
Epoch 5/40
410/410 [==============================] - 30s 73ms/step - batch: 204.5000 - size: 1.0000 - loss: 0.4911 - AUC: 0.8131 - val_loss: 0.4886 - val_AUC: 0.8182 - lr: 0.0010
Epoch 6/40
410/410 [==============================] - 30s 

In [10]:
model.optimizer.lr = 0.00002

In [11]:
import pandas as pd
historyFrame = pd.DataFrame(history.history)
historyFrame[["auc", "val_auc"]].plot()
historyFrame[["loss", "val_loss"]].plot()

KeyError: "None of [Index(['auc', 'val_auc'], dtype='object')] are in the [columns]"

In [ ]:
train_x.shape
# train_y